# SQL 고급 활용 및 튜닝

# 제 1장 아키텍처 기반 튜닝 원리

## 제1절 데이터베이스 아키텍처

## 1. 아키텍처 개관

### 가. ORACLE 아키텍처

- 데이터베이스 : 물리적인 디스크에 저장된 데이터집합 (데이터파일, 리두로그파일, 컨트롤파일)
- 인스턴스 : 공유메모리(SGA)와 이를 엑세스하는 프로세스 집합

#### 하나의 인스턴스는 하나의 데이터베이스를 액세스(Single)
#### 여러개의 인스턴스는 하나의 데이터베이스를 엑세스(RAC)

### 나. SQL Server 아키텍처

- 하나의 인스턴스당 최고 32,767개의 데이터베이스를 정의해서 사용
- 기본적으로 시스템데이터베이스가 만들어지면, 사용자데이터베이스를 추가하여 생성하는 구조

- 시스템데이터베이스 : mster, model, msdb, tempdb 등
- 사용자데이터베이스 : 데이터파일(mdf), 트랜잭션로그파일(ldf), 보조데이터파일(ndf)

## 2. 프로세스

- 서버프로세스 : 전면에 나서 사용자가 던지는 각종 명령을 처리
- 백그라운드프로세스 : 뒤에서 묵묵히 주어진 역할을 수행

### 가. 서버프로세스(Server Process)

- 사용자 프로세스와 통신하면서 사용자의 각종 명령어 처리
- ORACLE : 서버프로세스
- SQL Server : Worker thread

#### 처리절차
- 사용자의 요청
- SQL 파싱
- 커서를 열어서 SQL을 실행하면서 블록 READ
- 읽은 데이터를 정렬하여 요청한 결과집합을 만들어 네트워크를 통해 전송

#### 클라이언트가 서버프로세스와 연결하는 방식(예 오라클)

#### 전용서버 방식(Dedicated Server)
- 클라이언트 세션과 서버프로세스가 1:1 로 매핑
- 오라클의 가장 일반적인 방식
- 클라이언트 요청에 의해 리스너 프로세스는 dedicated server 생성
- 새로운 dedicated server 프로세스는 리스너에 의해 커넥션 권한을 상속받음
- 데이터베이스와 물리적인 커넥션을 맺음


1. (User)     연결요청                       ==> (Listener)
2. (Listnenr) 프로세스 생성 및 연결요청 상속 ==> (Server)
3. (Server)   RESEND 패킷 전송               ==> (User)
4. (User)     연결 후 작업 요청              ==> (Server)
5. (Server)   처리 후 결과 전송              ==> (User)
 

#### 공유서버 방식(Shared Server)
- 클라이언트 세션과 서버프로세스가 1:N로 매핑
- 클라이언트 요청에 의해 리스너 프로세스는 현재 사용가능한 dispatcher pool 탐색확인
- 리스너는 사용가능한 dispatcher 커넥션 정보를 클라이언트에 되돌려줌
- 클라이언트는 리스너 접속을 끝내고 바로 dispatcher로 접속

1. (User) 연결요청 ==> (Listener)
2. (Listener) 가용한 Dispatcher 포트번호 전송 ==> (User)
3. (User) 연결 후 작업 요청 ==> (Dispatcher)
4. (Dispatcher) 요청등록 ==> (SGA Request Queue)
5. (SGA Request Queue) 요청접수 ==> (Server)
6. (Server) 결과 등록 ==> (SGA Reponcse Queue)
7. (SGA Reponcse Queue) 결과수렴 ==> (Dispatcher)
8. (Dispatcher) 결과 전송 ==> (User)

### 나. 백그라운드프로세스

####              오라클                 ==                       SQL SERVER
**SMON(System Monitor) == Databese cleanup / Shrinking Thread**
- 장애가 발생한 시스템을 재기동할 때 인스턴스 복구를 수행하고, 임시 세그먼트와 익스텐트를 모니터링한다.

**PMON(Process Monitor) == Open data Services(OPS)**
- 이상이 생긴 프로세스가 사용하던 리소스를 복구한다.

**DBWn(Database Writers) == Lazywriter Thread**
- 버퍼 캐시에 있는 더티 버퍼를 데이터 파일에 기록

**LGWR(Log Writer) == Log writer Thread**
- 로그 버퍼 엔트리를 redo 로그 파일에 기록한다.

**ARCn(Archiver) == N/A**
- 꽉찬 리두로그가 덮어 쓰여지기 전에 archive로그 디렉토리로 백업한다.

**CKPT(Checkpoint) == Database Checkpoint Thread**
- checkpoint 프로세스는 이전의 checkpoint가 일어났던 마지막 시점 이후의 데이터베이스 변경 사항을 데이터파일에 기록하도록  
트리거링하고, 기록이 완료되면 현재 어디까지 기록했는지를 컨트롤 파일과 데이터 파일 헤더에 기록한다.
- 좀더 자세히 설명하면 write Ahead Logging 방식을 사용하는 DBMS는 리두로그에 기록해 둔 버퍼 브록에 대한 변경사항 중
현재 어디까지를 데이터 파일에 기록했는지 체크 포인트정보를 관리해야 한다.
- 이는 버퍼캐시와 데이터 파일이 동기화된 시점을 가리키며, 장애가 발생하면 마지막 체크포인트 이후 로그 데이터만 디스크에 
기록함으로써 인스턴스를 복구할 수 있도록 하는 용도로 사용된다.
- 이 정보를 갱신하는 주기가 길수록 장애 발생시 인스턴스 복구 시간도 길어진다.

**RECO(Recoverer) == Distributed Transaction Coordinator(DTC)**
- 분산 트랜잭션 과정에 발생한 문제를 해결한다.

## 제2절 SQL 파싱 부하

## 제3절 데이터베이스 Call과 네트워크 부하

## 제4절 데이터베이스 I/O 원리